In [1]:
#import the modules 
import os 
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
os.chdir('E:\\Desktop\\assignment\\supply_chain')

In [5]:
tweets = pd.read_excel("output.xlsx")## import the data
input_accounts = pd.read_csv("input_accounts_21_10_2022.csv")

In [8]:
#select English tweets
df_posts = tweets.loc[tweets['lang']=='en']

#import the module and create a list of stopwords
from nltk.corpus import stopwords

additional  = ['rt','rts','retweet','number','look','read','food','eat','amp','us','email','dm','pls','pleas',
              'hi','hey','team','thank','dmu','reach','take','drop','send','uk'] 
swords = set().union(stopwords.words('english'),additional)
#replace all signs and stopwords in tweets and tokenize tweets (tokenization means splitting up a larger body of text into pieces such as words, keywords, phrases, symbols and other elements called tokens; in our case we can split the tweets into words)
df_posts.drop_duplicates(subset='text',inplace=True)
df_posts.shape
df_posts['processed_text'] = df_posts['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ', regex=False)\
          .str.replace('(http\S+)', ' ', regex=False)\
          .str.replace('([^0-9a-z \t])',' ', regex=False)\
          .str.replace(' +',' ', regex=False)\
          .apply(lambda x: [i for i in x.split() if not i in swords])

#stem tweets to combine similar texts
from nltk.stem import PorterStemmer
ps = PorterStemmer()
df_posts['stemmed'] = df_posts['processed_text'].apply(lambda x: [ps.stem(i) for i in x if i != ''])
#word_count
df_posts['word_count'] = df_posts['stemmed'].apply(len)

#select  tweets published by JustEatTakeaway(JET)， Uber and Deliveroo 
JET = df_posts.loc[df_posts['user_id'].isin([272030125,21427907,25320756])]
Deliveroo = df_posts.loc[df_posts['user_id'].isin([884794020])]
Uber = df_posts.loc[df_posts['user_id'].isin([3100005042,1123948811691470849])]


#create dataframe only containts retweets 
JET_re = JET.loc[JET["replied_to_id"] != -1]
Deliveroo_re = Deliveroo.loc[Deliveroo["replied_to_id"] != -1]
Uber_re = Uber.loc[Uber["replied_to_id"] != -1]


#create dataframe only containts original posts
JET_or = JET.loc[JET["replied_to_id"] == -1]
Deliveroo_or = Deliveroo.loc[Deliveroo["replied_to_id"] == -1]
Uber_or = Uber.loc[Uber["replied_to_id"] == -1]

In [ ]:
#sentiment score 
import nltk.sentiment.vader as vd
from nltk import download
sia = vd.SentimentIntensityAnalyzer()
from nltk.tokenize import word_tokenize
df_posts['sentiment_score'] = df_posts['stemmed'].apply(lambda x: sum([ sia.polarity_scores(i)['compound'] for i in word_tokenize( ' '.join(x) )]) )

df_posts['sent_clasification'] = pd.cut(df_posts['sentiment_score'],\
          [-5,-2, 0, 2 , 5],\
          right=True,\
          include_lowest=True,\
          labels=['snegative', 'negative', 'positive', 'spositive'])

In [24]:
#CSR-RELATED
# define sustainability related content
df_posts['innovation'] = np.where(df_posts['text'].str.contains('colla|sustain|waste|plastic|recycle|pollutionResponsibility|Sustainable|Ethics|Accountability|Transparency|Environmental|Social|Charity|Donation|Community|Philanthropy|Giving|Volunteer|Empowerment|Equality|Diversity|Inclusion|Fairness|Ethical|Impactful|Green|Conservation|Sustainability|Humanitarian|Support|Corporate|Engagement|Initiative|Wellness|Stakeholder|Responsible|Welfare|Education|Contribution|Empathy|Empower|Non-profit|Resilience|Advocacy|Transparency|Respect|Partnerships|Progress|Collaboration|Empowering|Trust'), 1, 0)
df_posts['innovation']
#create sustainability tweets dataframe based on original tweets
innovation = df_posts.loc[df_posts['innovation']==1 ]
uninnovation = df_posts.loc[df_posts['innovation']!=1 ]

In [ ]:
#user engagement
df_posts['engagement']=df_posts['retweet_count']+df_posts['like_count']+df_posts['reply_count']+df_posts['quote_count']
df_posts['log_engage']=np.log(df_posts['engagement']+1)

In [ ]:
#CSR-RELATED
# define sustainability related content
df_posts['innovation'] = np.where(df_posts['text'].str.contains('colla|sustain|waste|plastic|recycle|pollutionResponsibility|Sustainable|Ethics|Accountability|Transparency|Environmental|Social|Charity|Donation|Community|Philanthropy|Giving|Volunteer|Empowerment|Equality|Diversity|Inclusion|Fairness|Ethical|Impactful|Green|Conservation|Sustainability|Humanitarian|Support|Corporate|Engagement|Initiative|Wellness|Stakeholder|Responsible|Welfare|Education|Contribution|Empathy|Empower|Non-profit|Resilience|Advocacy|Transparency|Respect|Partnerships|Progress|Collaboration|Empowering|Trust'), 1, 0)
df_posts['innovation']
#create sustainability tweets dataframe based on original tweets
innovation = df_posts.loc[df_posts['innovation']==1 ]
uninnovation = df_posts.loc[df_posts['innovation']!=1 ]

In [21]:
#re
df_posts.loc[df_posts['replied_to_id'] == -1, 're'] = 1
df_posts.loc[df_posts['replied_to_id'] != -1, 're'] = 0




C:\Users\Jiang Bowen\AppData\Local\Temp\ipykernel_40884\2502972285.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JET['innovation'] = np.where(JET['text'].str.contains('colla|sustain|waste|plastic|recycle|pollutionResponsibility|Sustainable|Ethics|Accountability|Transparency|Environmental|Social|Charity|Donation|Community|Philanthropy|Giving|Volunteer|Empowerment|Equality|Diversity|Inclusion|Fairness|Ethical|Impactful|Green|Conservation|Sustainability|Humanitarian|Support|Corporate|Engagement|Initiative|Wellness|Stakeholder|Responsible|Welfare|Education|Contribution|Empathy|Empower|Non-profit|Resilience|Advocacy|Transparency|Respect|Partnerships|Progress|Collaboration|Empowering|Trust'), 1, 0)


In [43]:
#create year,month,hour for charts  
df_posts['year'] = pd.to_datetime(df_posts['created_at']).dt.year
df_posts['month'] = pd.to_datetime(df_posts['created_at']).dt.month
df_posts['hour'] = pd.to_datetime(df_posts['created_at']).dt.hour

df_posts['busy'] = np.where(df_posts['hour'].isin([14, 16, 15, 17, 18, 19]), 1, 0)


In [99]:
#create sustainability tweets dataframe based on original tweets
innovation = df_posts.loc[df_posts['innovation']==1 ]
uninnovation = df_posts.loc[df_posts['innovation']!=1 ]


JET = df_posts.loc[df_posts['user_id'].isin([272030125,21427907,25320756])]
Deliveroo = df_posts.loc[df_posts['user_id'].isin([884794020])]
Uber = df_posts.loc[df_posts['user_id'].isin([3100005042,1123948811691470849])]


#create dataframe only containts retweets 
JET_re = JET.loc[JET["replied_to_id"] != -1]
Deliveroo_re = Deliveroo.loc[Deliveroo["replied_to_id"] != -1]
Uber_re = Uber.loc[Uber["replied_to_id"] != -1]


#create dataframe only containts original posts
JET_or = JET.loc[JET["replied_to_id"] == -1]
Deliveroo_or = Deliveroo.loc[Deliveroo["replied_to_id"] == -1]
Uber_or = Uber.loc[Uber["replied_to_id"] == -1]



In [116]:
import pandas as pd
import statsmodels.formula.api as smf
from docx import Document
from tabulate import tabulate

# Perform linear regression
results = smf.ols('log_engage ~ similarity + sentiment_score + re + word_count + innovation + busy', data=df_posts).fit()

# Get the statistical summary of the regression results
summary_table = results.summary().tables[1]
summary_data = [list(map(str, row)) for row in summary_table.data[1:]]

# Create a DataFrame to store the table data
df_summary = pd.DataFrame(summary_data, columns=summary_table.data[0])

# Create a Word document
doc = Document()

# Add the regression results to the Word document
doc.add_paragraph('Linear Regression Results')
doc.add_paragraph(tabulate(df_summary, headers='keys', tablefmt='grid'))

# Save the Word document
doc.save('linear_regression_results.docx')

print("Linear regression results saved to 'linear_regression_results.docx'")


Linear regression results saved to 'linear_regression_results.docx'


In [64]:
results = smf.ols('log_engage ~ similirity+sentiment_score+re+word_count+innovation+busy ', data=innovation).fit()
print(results.summary())
from docx import Document


results_df = results.summary().tables[1].as_html()
df = pd.read_html(results_df)[0]
df.to_excel('regression_results.xlsx', index=False)

                            OLS Regression Results                            
Dep. Variable:             log_engage   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     54.11
Date:                Tue, 06 Jun 2023   Prob (F-statistic):           4.94e-39
Time:                        21:26:17   Log-Likelihood:                -339.98
No. Observations:                 282   AIC:                             692.0
Df Residuals:                     276   BIC:                             713.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4199      0.167     

In [57]:
treatment_group = df_posts[df_posts['innovation'] == 1]
control_group = df_posts[df_posts['innovation'] == 0]


treatment_results = smf.ols('log_engage ~ similirity+sentiment_score+re+word_count+busy', data=treatment_group).fit()
control_results = smf.ols('log_engage ~ similirity+sentiment_score+re+word_count+busy', data=control_group).fit()

cate_results = treatment_results.params - control_results.params

print(cate_results)

Intercept          0.732237
similirity        -1.328147
sentiment_score   -0.355364
re                -4.571168
word_count         0.052372
busy               0.461925
dtype: float64


In [96]:


user_ids = [272030125, 21427907, 25320756, 884794020,3100005042, 1123948811691470000]
output_file = "user_stats.xlsx"

with pd.ExcelWriter(output_file) as writer:
    for user_id in user_ids:
        user_data = df_posts[df_posts['user_id'] == user_id][['log_engage', 'similirity', 'sentiment_score', 're', 'word_count', 'innovation', 'busy']]
        descriptive_stats = user_data.describe()
        descriptive_stats.to_excel(writer, sheet_name=f"User_{user_id}")

print("Descriptive statistics saved to user_stats.xlsx")

Descriptive statistics saved to user_stats.xlsx


In [ ]:
df_posts.rename(columns={'similirity': 'similarity', 'innovation': 'is_CSR', 'innovation': 'is_CSR'}, inplace=True)

In [74]:
df_posts = pd.read_excel("df_posts.xlsx")

In [108]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

variables = ['similarity', 'sentiment_score', 're', 'word_count', 'innovation', 'busy']

# Remove missing values for specified variables
df_posts.dropna(subset=variables, inplace=True)

# Prepare feature matrix X and target variable log_engage
X = df_posts[['similarity', 'sentiment_score', 're', 'word_count', 'innovation', 'busy']]
y = df_posts['log_engage']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# Fit the Random Forest model on the training set
rf_model.fit(X_train, y_train)
# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = np.mean(np.abs(y_test - y_pred_rf))
correlation_rf = np.corrcoef(y_test, y_pred_rf)[0, 1]

# Output results
print("RF R2 Score:", r2_rf)
print("RF RMSE:", rmse_rf)
print("RF MAE:", mae_rf)
print("RF Correlation Coefficient:", correlation_rf)

# Create an AdaBoosting regression model
ada_model = AdaBoostRegressor(n_estimators=100, random_state=42)
# Fit the AdaBoosting model on the training set
ada_model.fit(X_train, y_train)
# Predict on the test set
y_pred_ada = ada_model.predict(X_test)

r2_ada = r2_score(y_test, y_pred_ada)
rmse_ada = np.sqrt(mean_squared_error(y_test, y_pred_ada))
mae_ada = np.mean(np.abs(y_test - y_pred_ada))
correlation_ada = np.corrcoef(y_test, y_pred_ada)[0, 1]

# Output results
print("Ada R2 Score:", r2_ada)
print("Ada RMSE:", rmse_ada)
print("Ada MAE:", mae_ada)
print("Ada Correlation Coefficient:", correlation_ada)

# Create a GBRT regression model
gbrt_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
# Fit the GBRT model on the training set
gbrt_model.fit(X_train, y_train)
# Predict on the test set
y_pred_gbrt = gbrt_model.predict(X_test)

r2_gbrt = r2_score(y_test, y_pred_gbrt)
rmse_gbrt = np.sqrt(mean_squared_error(y_test, y_pred_gbrt))
mae_gbrt = np.mean(np.abs(y_test - y_pred_gbrt))
correlation_gbrt = np.corrcoef(y_test, y_pred_gbrt)[0, 1]

# Output results
print("GBRT R2 Score:", r2_gbrt)
print("GBRT RMSE:", rmse_gbrt)
print("GBRT MAE:", mae_gbrt)
print("GBRT Correlation Coefficient:", correlation_gbrt)

# Create a Bagging regression model
bagging_model = BaggingRegressor(n_estimators=100, random_state=42)
# Fit the Bagging model on the training set
bagging_model.fit(X_train, y_train)
# Predict on the test set
y_pred_bagging = bagging_model.predict(X_test)

r2_bagging = r2_score(y_test, y_pred_bagging)
rmse_bagging = np.sqrt(mean_squared_error(y_test, y_pred_bagging))
mae_bagging = np.mean(np.abs(y_test - y_pred_bagging))
correlation_bagging = np.corrcoef(y_test, y_pred_bagging)[0, 1]

# Output results
print("BAG R2 Score:", r2_bagging)
print("BAG RMSE:", rmse_bagging)
print("BAG MAE:", mae_bagging)
print("BAG Correlation Coefficient:", correlation_bagging)

# Create a MLP regression model (neural network)
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 100), activation='relu', random_state=42)
# Fit the MLP model on the training set
mlp_model.fit(X_train, y_train)
# Predict on the test set
y_pred_mlp = mlp_model.predict(X_test)

r2_mlp = r2_score(y_test, y_pred_mlp)
rmse_mlp = np.sqrt(mean_squared_error(y_test, y_pred_mlp))
mae_mlp = np.mean(np.abs(y_test - y_pred_mlp))
correlation_mlp = np.corrcoef(y_test, y_pred_mlp)[0, 1]

# Output results
print("MLP R2 Score:", r2_mlp)
print("MLP RMSE:", rmse_mlp)
print("MLP MAE:", mae_mlp)
print("MLP Correlation Coefficient:", correlation_mlp)


RF R2 Score: 0.39559319159203876
RF RMSE: 0.9789760411785597
RF MAE: 0.699327376875404
RF Correlation Coefficient: 0.6393154063234263
Ada R2 Score: 0.39532646397167237
Ada RMSE: 0.9791920307605853
Ada MAE: 0.7430054814383398
Ada Correlation Coefficient: 0.637625794875185
GBRT R2 Score: 0.44037990329454624
GBRT RMSE: 0.9420068057436769
GBRT MAE: 0.6952706467263758
GBRT Correlation Coefficient: 0.6636216872493221
BAG R2 Score: 0.3947623904673523
BAG RMSE: 0.9796486470127829
BAG MAE: 0.7001577978294486
BAG Correlation Coefficient: 0.6387803773939683
MLP R2 Score: 0.42160557565902324
MLP RMSE: 0.9576778399572058
MLP MAE: 0.7128945037549868
MLP Correlation Coefficient: 0.6505831706728901


In [110]:
importances_rf = rf_model.feature_importances_
print("RF Feature Importances:")
for feature, importance in zip(X.columns, importances_rf):
    print(feature, ":", importance)

# Output feature importance scores of the GBRT model
importances_gbrt = gbrt_model.feature_importances_
print("GBRT Feature Importances:")
for feature, importance in zip(X.columns, importances_gbrt):
    print(feature, ":", importance)


RF Feature Importances:
similirity : 0.23344426320258369
sentiment_score : 0.1590813780925364
re : 0.4101056269030394
word_count : 0.17054469948978832
innovation : 0.005734261209020443
busy : 0.021089771103031736
GBRT Feature Importances:
similirity : 0.012945646898462547
sentiment_score : 0.045236702331010535
re : 0.7695251443295478
word_count : 0.1532522813982502
innovation : 0.0004979426859810387
busy : 0.01854228235674787


In [122]:
df_posts.rename(columns={'similirity': 'similarity', 'innovation': 'is_CSR', 'innovation': 'is_CSR'}, inplace=True)
df_posts['year'] = pd.to_datetime(df_posts['year'], format='%Y')
df_posts['month'] = pd.to_datetime(df_posts['month'], format='%m')
df_posts['is_pandemic'] = 0
df_posts.loc[(df_posts['year'] >= pd.to_datetime('2021')) & (df_posts['month'] >= pd.to_datetime('2021-01')), 'is_pandemic'] = 1


In [138]:
df_posts.rename(columns={'re': 'is_re','busy':'busy_time'}, inplace=True)

In [127]:
df_posts['is_pandemic'] = (df_posts['year'].dt.year >= 2021).astype(int)

In [129]:
pandemic_counts = df_posts['is_pandemic'].value_counts()
print(pandemic_counts)

1    11716
0     2258
Name: is_pandemic, dtype: int64


In [133]:
import pandas as pd
import statsmodels.formula.api as smf
from docx import Document
from tabulate import tabulate

# Perform linear regression
results = smf.ols('log_engage ~ similarity + sentiment_score + re + word_count + is_CSR + busy + is_pandemic', data=df_posts).fit()

# Get the statistical summary of the regression results
summary_table = results.summary().tables[1]
summary_data = [list(map(str, row)) for row in summary_table.data[1:]]

# Create a DataFrame to store the table data
df_summary = pd.DataFrame(summary_data, columns=summary_table.data[0])

# Create a Word document
doc = Document()

# Add the regression results to the Word document
doc.add_paragraph('Linear Regression Results')
doc.add_paragraph(tabulate(df_summary, headers='keys', tablefmt='grid'))

# Save the Word document
doc.save('linear_regression_results.docx')

print("Linear regression results saved to 'linear_regression_results.docx'")


Linear regression results saved to 'linear_regression_results.docx'


In [142]:
results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time+ is_pandemic', data=df_posts).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             log_engage   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     1591.
Date:                Thu, 08 Jun 2023   Prob (F-statistic):               0.00
Time:                        15:30:05   Log-Likelihood:                -19212.
No. Observations:               13974   AIC:                         3.844e+04
Df Residuals:                   13967   BIC:                         3.849e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.3656      0.035     

In [149]:
treatment_group = df_posts[df_posts['is_CSR'] == 1]
control_group = df_posts[df_posts['is_CSR'] == 0]

# Regression analysis for the treatment group
treatment_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=treatment_group).fit()

# Regression analysis for the control group
control_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=control_group).fit()

# Summarize the regression results
results_summary = summary_col([treatment_results, control_results], stars=True)

# Print the results to the console
print(results_summary)



                log_engage I log_engage II
------------------------------------------
Intercept       -0.0616      0.3731***    
                (0.2017)     (0.0355)     
similarity      -0.0146      0.0208***    
                (0.0620)     (0.0077)     
sentiment_score -0.0193      -0.1494***   
                (0.0555)     (0.0175)     
is_re           2.0564***    2.2388***    
                (0.1480)     (0.0254)     
word_count      0.0261***    -0.0194***   
                (0.0079)     (0.0015)     
busy_time       -0.0290      0.1931***    
                (0.0992)     (0.0165)     
is_pandemic     0.5261***    0.4378***    
                (0.1304)     (0.0267)     
R-squared       0.5232       0.3942       
R-squared Adj.  0.5128       0.3939       
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [143]:
results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time+ is_pandemic', data=control_group).fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:             log_engage   R-squared:                       0.394
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     1484.
Date:                Thu, 08 Jun 2023   Prob (F-statistic):               0.00
Time:                        15:58:47   Log-Likelihood:                -18833.
No. Observations:               13692   AIC:                         3.768e+04
Df Residuals:                   13685   BIC:                         3.773e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.3731      0.035     

In [154]:
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
import scipy.stats as stats

# Perform regression analysis for the treatment group
treatment_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=treatment_group).fit()

# Perform regression analysis for the control group
control_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=control_group).fit()

# Summarize the regression results
results_summary = summary_col([treatment_results, control_results], stars=True)

# Print the results to the console
print(results_summary)

# Extract coefficients and standard errors for the treatment group
treatment_coefs = treatment_results.params
treatment_std_errors = treatment_results.bse

# Extract coefficients and standard errors for the control group
control_coefs = control_results.params
control_std_errors = control_results.bse

# Calculate the difference in coefficients between treatment and control groups
coef_diff = treatment_coefs - control_coefs

# Perform a statistical test to determine the significance of the differences
t_values = coef_diff / np.sqrt(treatment_std_errors**2 + control_std_errors**2)
p_values = 2 * (1 - stats.t.cdf(np.abs(t_values), df=len(treatment_group) + len(control_group) - len(coef_diff)))

# Create a DataFrame to display the results
ate_results = pd.DataFrame({'Coefficient Difference': coef_diff, 'Standard Error (Treatment)': treatment_std_errors, 'Standard Error (Control)': control_std_errors, 'T-value': t_values, 'P-value': p_values})

# Print the ATE results
print(ate_results)



                log_engage I log_engage II
------------------------------------------
Intercept       -0.0616      0.3731***    
                (0.2017)     (0.0355)     
similarity      -0.0146      0.0208***    
                (0.0620)     (0.0077)     
sentiment_score -0.0193      -0.1494***   
                (0.0555)     (0.0175)     
is_re           2.0564***    2.2388***    
                (0.1480)     (0.0254)     
word_count      0.0261***    -0.0194***   
                (0.0079)     (0.0015)     
busy_time       -0.0290      0.1931***    
                (0.0992)     (0.0165)     
is_pandemic     0.5261***    0.4378***    
                (0.1304)     (0.0267)     
R-squared       0.5232       0.3942       
R-squared Adj.  0.5128       0.3939       
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
                 Coefficient Difference  Standard Error (Treatment)  \
Intercept                     -0.434663                    0.201663   
similarity              

In [155]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Create a new column 'treatment' and set treatment group to 1 and control group to 0 based on 'is_CSR'
df_posts['treatment'] = np.where(df_posts['is_CSR'] == 1, 1, 0)

# Extract variables for matching
matching_variables = ['similarity', 'sentiment_score']

# Perform propensity score matching
matcher = NearestNeighbors(n_neighbors=1)
matcher.fit(df_posts[matching_variables])
distances, indices = matcher.kneighbors(df_posts[matching_variables])

# Extract indices of matched samples
matched_indices = indices[:, 0]

# Extract data for matched treatment and control groups
matched_treatment_group = df_posts.loc[df_posts.index.isin(matched_indices) & (df_posts['treatment'] == 1)]
matched_control_group = df_posts.loc[df_posts.index.isin(matched_indices) & (df_posts['treatment'] == 0)]

# Perform CATE analysis
cate_results = smf.ols('log_engage ~ similarity + sentiment_score', data=matched_treatment_group).fit()

# Print CATE results
print(cate_results.summary())


                            OLS Regression Results                            
Dep. Variable:             log_engage   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.611
Date:                Thu, 08 Jun 2023   Prob (F-statistic):              0.202
Time:                        17:40:01   Log-Likelihood:                -329.43
No. Observations:                 221   AIC:                             664.9
Df Residuals:                     218   BIC:                             675.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           2.4182      0.228     

In [157]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Create a new column 'treatment' and set treatment group to 1 and control group to 0 based on 'is_CSR'
df_posts['treatment'] = np.where(df_posts['is_CSR'] == 1, 1, 0)

# Extract variables for matching
matching_variables = ['sentiment_score', 'is_re', 'word_count', 'busy_time', 'is_pandemic']

# Perform propensity score matching
matcher = NearestNeighbors(n_neighbors=1)
matcher.fit(df_posts[matching_variables])
distances, indices = matcher.kneighbors(df_posts[matching_variables])

# Extract indices of matched samples
matched_indices = indices[:, 0]

# Extract data for matched treatment and control groups
matched_treatment_group = df_posts.loc[df_posts.index.isin(matched_indices) & (df_posts['treatment'] == 1)]
matched_control_group = df_posts.loc[df_posts.index.isin(matched_indices) & (df_posts['treatment'] == 0)]

# Perform regression analysis for treatment group
treatment_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=matched_treatment_group).fit()

# Perform regression analysis for control group
control_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=matched_control_group).fit()

# Summarize regression results
results_summary = summary_col([treatment_results, control_results], stars=True)

# Print results to console
print(results_summary)

# Extract coefficients and standard errors for treatment and control groups
treatment_coefs = treatment_results.params
treatment_std_errors = treatment_results.bse

control_coefs = control_results.params
control_std_errors = control_results.bse

# Calculate the difference in coefficients between treatment and control groups
coef_diff = treatment_coefs - control_coefs

# Perform statistical test to determine the significance of the differences
t_values = coef_diff / np.sqrt(treatment_std_errors**2 + control_std_errors**2)
p_values = 2 * (1 - stats.t.cdf(np.abs(t_values), df=len(matched_treatment_group) + len(matched_control_group) - len(coef_diff)))

# Create DataFrame to display results
ate_results = pd.DataFrame({'Coefficient Difference': coef_diff, 'Standard Error (Treatment)': treatment_std_errors, 'Standard Error (Control)': control_std_errors, 'T-value': t_values, 'P-value': p_values})

# Print ATE results
print(ate_results)



                log_engage I log_engage II
------------------------------------------
Intercept       0.0439       0.1754***    
                (0.2787)     (0.0654)     
similarity      -0.0889      0.0129       
                (0.0753)     (0.0168)     
sentiment_score -0.0523      -0.0512*     
                (0.0547)     (0.0291)     
is_re           2.3001***    2.3347***    
                (0.1873)     (0.0411)     
word_count      0.0183*      -0.0082***   
                (0.0097)     (0.0026)     
busy_time       0.0647       0.1964***    
                (0.1104)     (0.0338)     
is_pandemic     0.6376***    0.6064***    
                (0.1559)     (0.0424)     
R-squared       0.5048       0.4506       
R-squared Adj.  0.4882       0.4498       
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
                 Coefficient Difference  Standard Error (Treatment)  \
Intercept                     -0.131418                    0.278720   
similarity              

In [158]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Create a new column 'treatment' and set treatment group to 1 and control group to 0 based on 'is_CSR'
df_posts['treatment'] = np.where(df_posts['is_CSR'] == 1, 1, 0)

# Extract variables for matching
matching_variables = ['sentiment_score', 'is_re', 'word_count', 'busy_time', 'is_pandemic']

# Perform propensity score matching
matcher = NearestNeighbors(n_neighbors=1)
matcher.fit(df_posts[matching_variables])
distances, indices = matcher.kneighbors(df_posts[matching_variables])

# Extract indices of matched samples
matched_indices = indices[:, 0]

# Extract data for matched treatment and control groups
matched_treatment_group = df_posts.loc[df_posts.index.isin(matched_indices) & (df_posts['treatment'] == 1)]
matched_control_group = df_posts.loc[df_posts.index.isin(matched_indices) & (df_posts['treatment'] == 0)]

# Perform regression analysis for treatment group
treatment_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=matched_treatment_group).fit()

# Perform regression analysis for control group
control_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=matched_control_group).fit()

# Summarize regression results
results_summary = summary_col([treatment_results, control_results], stars=True)

# Print results to console
print(results_summary)

# Extract coefficients and standard errors for treatment and control groups
treatment_coefs = treatment_results.params
treatment_std_errors = treatment_results.bse

control_coefs = control_results.params
control_std_errors = control_results.bse

# Calculate the difference in coefficients between treatment and control groups
coef_diff = treatment_coefs - control_coefs

# Perform statistical test to determine the significance of the differences
t_values = coef_diff / np.sqrt(treatment_std_errors**2 + control_std_errors**2)
p_values = 2 * (1 - stats.t.cdf(np.abs(t_values), df=len(matched_treatment_group) + len(matched_control_group) - len(coef_diff)))

# Create DataFrame to display results
ate_results = pd.DataFrame({'Coefficient Difference': coef_diff, 'Standard Error (Treatment)': treatment_std_errors, 'Standard Error (Control)': control_std_errors, 'T-value': t_values, 'P-value': p_values})

# Print CATE results
print(ate_results)



                log_engage I log_engage II
------------------------------------------
Intercept       -0.0616      0.1754***    
                (0.2017)     (0.0654)     
similarity      -0.0146      0.0129       
                (0.0620)     (0.0168)     
sentiment_score -0.0193      -0.0512*     
                (0.0555)     (0.0291)     
is_re           2.0564***    2.3347***    
                (0.1480)     (0.0411)     
word_count      0.0261***    -0.0082***   
                (0.0079)     (0.0026)     
busy_time       -0.0290      0.1964***    
                (0.0992)     (0.0338)     
is_pandemic     0.5261***    0.6064***    
                (0.1304)     (0.0424)     
R-squared       0.5232       0.4506       
R-squared Adj.  0.5128       0.4498       
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
                 Coefficient Difference  Standard Error (Treatment)  \
Intercept                     -0.236916                    0.201663   
similarity              

In [161]:
df_posts.to_csv('df_posts.csv', index=False)

In [6]:
df_posts = pd.read_csv("df_posts0611.csv")## import the data

In [162]:
df_posts['is_JET'] = df_posts['user_id'].isin([272030125, 21427907, 25320756]).astype(int)
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
import scipy.stats as stats
treatment_group = df_posts[df_posts['is_JET'] == 1]
control_group = df_posts[df_posts['is_JET'] == 0]

# Perform regression analysis for the treatment group
treatment_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=treatment_group).fit()

# Perform regression analysis for the control group
control_results = smf.ols('log_engage ~ similarity + sentiment_score + is_re + word_count + busy_time + is_pandemic', data=control_group).fit()

# Summarize the regression results
results_summary = summary_col([treatment_results, control_results], stars=True)

# Print the results to the console
print(results_summary)

# Extract coefficients and standard errors for the treatment group
treatment_coefs = treatment_results.params
treatment_std_errors = treatment_results.bse

# Extract coefficients and standard errors for the control group
control_coefs = control_results.params
control_std_errors = control_results.bse

# Calculate the difference in coefficients between the treatment and control groups
coef_diff = treatment_coefs - control_coefs

# Perform a statistical test to determine the significance of the differences
t_values = coef_diff / np.sqrt(treatment_std_errors**2 + control_std_errors**2)
p_values = 2 * (1 - stats.t.cdf(np.abs(t_values), df=len(treatment_group) + len(control_group) - len(coef_diff)))

# Create a DataFrame to display the results
ate_results = pd.DataFrame({'Coefficient Difference': coef_diff, 'Standard Error (Treatment)': treatment_std_errors, 'Standard Error (Control)': control_std_errors, 'T-value': t_values, 'P-value': p_values})

# Print the ATE results
print(ate_results)



                log_engage I log_engage II
------------------------------------------
Intercept       -0.0812*     0.3359***    
                (0.0431)     (0.0601)     
similarity      0.0277**     0.0214**     
                (0.0111)     (0.0096)     
sentiment_score 0.0048       -0.2563***   
                (0.0190)     (0.0243)     
is_re           1.9355***    2.2608***    
                (0.0312)     (0.0371)     
word_count      0.0227***    -0.0315***   
                (0.0019)     (0.0021)     
busy_time       -0.0119      0.2433***    
                (0.0208)     (0.0222)     
is_pandemic     0.0692**     0.6896***    
                (0.0294)     (0.0522)     
R-squared       0.6319       0.3490       
R-squared Adj.  0.6314       0.3485       
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
                 Coefficient Difference  Standard Error (Treatment)  \
Intercept                     -0.417080                    0.043062   
similarity              

In [7]:
import pandas as pd

output_file = "user_stats.xlsx"

with pd.ExcelWriter(output_file) as writer:
    for is_csr_value in [1, 0]:
        user_data = df_posts[df_posts['is_CSR'] == is_csr_value][['log_engage', 'similarity', 'sentiment_score', 'is_re', 'word_count', 'busy_time', 'is_pandemic']]
        descriptive_stats = user_data.describe()
        descriptive_stats.to_excel(writer, sheet_name=f"is_CSR_{is_csr_value}")

    all_data = df_posts[['log_engage', 'similarity', 'sentiment_score', 'is_re', 'word_count', 'busy_time', 'is_pandemic']]
    all_descriptive_stats = all_data.describe()
    all_descriptive_stats.to_excel(writer, sheet_name="All_Data")

print("Descriptive statistics saved to user_stats.xlsx")


Descriptive statistics saved to user_stats.xlsx


In [8]:
import pandas as pd

output_file = "user_stats_JET.xlsx"

with pd.ExcelWriter(output_file) as writer:
    for is_csr_value in [1, 0]:
        user_data = df_posts[df_posts['is_JET'] == is_csr_value][['log_engage', 'similarity', 'sentiment_score', 'is_re', 'word_count', 'busy_time', 'is_pandemic']]
        descriptive_stats = user_data.describe()
        descriptive_stats.to_excel(writer, sheet_name=f"is_JET{is_csr_value}")

    all_data = df_posts[['log_engage', 'similarity', 'sentiment_score', 'is_re', 'word_count', 'busy_time', 'is_pandemic']]
    all_descriptive_stats = all_data.describe()
    all_descriptive_stats.to_excel(writer, sheet_name="All_Data")

print("Descriptive statistics saved to user_stats.xlsx")


Descriptive statistics saved to user_stats.xlsx
